# MLflow Models:

An MLflow Model is a standard format for packaging machine learning models that can be used in a variety of downstream tools. It lets you save a model in different “flavors” that can be understood by different downstream tools.

## What is a MLflow Flavor?

Flavors are the key concept that makes MLflow Models powerful: they are a convention that deployment tools can use to understand the model, which makes it possible to write tools that work with models from any ML library without having to integrate each tool with each library. 

## Built-In Model Flavors

* Python Function (python_function)

* R Function (crate)

* H2O (h2o)

* Keras (keras)

* MLeap (mleap)

* PyTorch (pytorch)

* Scikit-learn (sklearn)

* Spark MLlib (spark)

* TensorFlow (tensorflow)

* ONNX (onnx)

* MXNet Gluon (gluon)

* XGBoost (xgboost)

* LightGBM (lightgbm)

* CatBoost (catboost)

* Spacy(spaCy)

* Fastai(fastai)

* Statsmodels (statsmodels)

* Prophet (prophet)

* Pmdarima (pmdarima)

* OpenAI (openai) (Experimental)

* LangChain (langchain) (Experimental)

* John Snow Labs (johnsnowlabs) (Experimental)

* Diviner (diviner)

* Transformers (transformers) (Experimental)

* SentenceTransformers (sentence_transformers) (Experimental)

* Promptflow (promptflow) (Experimental)



## MLmodel file

```python
# Directory written by mlflow.sklearn.save_model(model, "my_model")
my_model/
├── MLmodel
├── model.pkl
├── conda.yaml
├── python_env.yaml
└── requirements.txt
```

## Sklearn Flavor

The mlflow.sklearn module provides an API for logging and loading scikit-learn models. This module exports scikit-learn models with the following flavors:

In [1]:
# Create experiment
import mlflow 
name = "mlflow-flavors"
try:
    experiment = mlflow.create_experiment(
        name=name,
        tags={
            "project_name": "logging_models",
            "topic":"run_management",
            "mlflow.note.content": "This experiment is used to log sklearn models"
        }
    )
except:
    print("Experiment already exists")
    experiment = mlflow.get_experiment_by_name(name)

experiment = mlflow.set_experiment(name)

Experiment already exists


In [2]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=10)

with mlflow.start_run(run_name="logging-random-forest") as run:
    mlflow.sklearn.log_model(sk_model = rfc, artifact_path = "random-forest-model")

## Model With Signature

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from mlflow.models.signature import infer_signature
import mlflow 

In [4]:
# load the iris dataset
iris = load_iris(as_frame=True)
x = iris.data
y = iris.target

# infer signature
signature = infer_signature(model_input=x, model_output=y)

c:\Users\manue\projects\mlflow_for_ml_dev\.venv\Lib\site-packages\mlflow\types\utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [5]:
for input in signature.inputs:
    print(input)

'sepal length (cm)': double (required)
'sepal width (cm)': double (required)
'petal length (cm)': double (required)
'petal width (cm)': double (required)


In [6]:
for input in signature.outputs:
    print(input)

'target': integer (required)


In [7]:
rfc = RandomForestClassifier()

rfc.fit(x, y)

with mlflow.start_run(run_name="logging_with_signature", experiment_id=experiment.experiment_id):
    # Log the model
    mlflow.sklearn.log_model(sk_model=rfc, artifact_path = rfc.__class__.__name__, signature=signature)

c:\Users\manue\projects\mlflow_for_ml_dev\.venv\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\manue\projects\mlflow_for_ml_dev\.venv\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Logging with input example

In [9]:
# get a sample input
input_example = x.iloc[0:10]
print(input_example)

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2
5                5.4               3.9                1.7               0.4
6                4.6               3.4                1.4               0.3
7                5.0               3.4                1.5               0.2
8                4.4               2.9                1.4               0.2
9                4.9               3.1                1.5               0.1


In [13]:
rfc = RandomForestClassifier()

with mlflow.start_run(run_name="logging_with_input_example", experiment_id=experiment.experiment_id): 
    # Log the model
    mlflow.sklearn.log_model(
        sk_model = rfc,
        artifact_path = rfc.__class__.__name__,
        signature = signature,
        input_example = input_example
    )

c:\Users\manue\projects\mlflow_for_ml_dev\.venv\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\manue\projects\mlflow_for_ml_dev\.venv\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Logging with code paths 

In [27]:
from mlflow_for_ml_dev.utils.utils import get_root_project
# get a sample input
input_example = x.iloc[0:10]
project_path = get_root_project()
rfc = RandomForestClassifier()
print((project_path / "mlflow_for_ml_dev").as_posix())
with mlflow.start_run(run_name="logging_with_code_paths", experiment_id=experiment.experiment_id):
    # Log the model
    mlflow.sklearn.log_model(
        sk_model=rfc,
        artifact_path = rfc.__class__.__name__,  
        signature=signature,
        input_example=input_example,
        code_paths=[(project_path / "mlflow_for_ml_dev" / "experiments" /"exp_utils.py").as_posix()]
    )

C:/Users/manue/projects/mlflow_for_ml_dev/mlflow_for_ml_dev


c:\Users\manue\projects\mlflow_for_ml_dev\.venv\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\manue\projects\mlflow_for_ml_dev\.venv\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Registering the Model

In [ ]:
registered_model_name = "iris_rfc"

# get a sample input
input_example = x.iloc[0:10]
rfc = RandomForestClassifier()

with mlflow.start_run(run_name="logging_and_registering", experiment_id=experiment.experiment_id):

    # Log the model
    mlflow.sklearn.log_model(
        sk_model=rfc,
        artifact_path = rfc.__class__.__name__,
        signature=signature,
        input_example=input_example,
        code_paths=[(project_path / "mlflow_for_ml_dev").as_posix()],
        registered_model_name=registered_model_name
    )